In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
glove_model = load_glove_model()

NameError: name 'load_glove_model' is not defined

In [1]:
glove_model.shape

NameError: name 'glove_model' is not defined

In [21]:
import glob
from dataset import ImageCaption
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from models import CNN
import numpy as np
from torchtext.data.metrics import bleu_score
from utils import *

input_size = 224

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_cap_dataset_train = ImageCaption("image_captions/image_mapping and captions/12/image_names.txt", "image_captions/image_mapping and captions/12/captions.txt",data_transforms["train"],"train")
image_cap_dataset_test = ImageCaption("image_captions/image_mapping and captions/12/image_names.txt", "image_captions/image_mapping and captions/12/captions.txt",data_transforms["val"],"val")

image_cap_train_dataloader  = DataLoader(image_cap_dataset_train, batch_size=1,num_workers=10, shuffle=True)
image_capt_test_dataloader  = DataLoader(image_cap_dataset_test, batch_size=1,num_workers=10, shuffle=False)





In [26]:
for image,tokens in image_cap_train_dataloader:
    print(tokens)

[('Girl',), ('smiling',), ('for',), ('camera',), ('with',), ('boy',), ('giving',), ('her',), ('bunny',), ('ears',)]
[('Concentration',), ('campsite',), ('.',)]
[('Man',), ('on',), ('a',), ('motorcycle',), ('performing',), ('a',), ('trick',), ('high',), ('in',), ('the',), ('air',), ('.',)]
[('Shirtless',), ('men',), ('hike',), ('through',), ('a',), ('canyon',), ('.',)]
[('A',), ('tan',), ('long',), ('-',), ('haired',), ('dog',), ('is',), ('running',), ('in',), ('the',), ('forest',)]
[('Young',), ('girl',), ('sliding',), ('down',), ('an',), ('inflated',), ('slide',)]
[('Two',), ('skiers',), ('getting',), ('ready',), ('to',), ('jump',), ('off',), ('of',), ('a',), ('ski',), ('lift',), ('.',)]
[('Two',), ('young',), ('boys',), ('playing',), ('on',), ('a',), ('white',), ('sofa',), ('.',)]
[('Two',), ('children',), ('ride',), ('saucers',), ('down',), ('a',), ('snowy',), ('hill',), ('.',)]
[('A',), ('white',), ('dog',), ('carries',), ('a',), ('bottle',), ('in',), ('its',), ('mouth',), ('throug

KeyboardInterrupt: 

One person is tumbling on the ground next to another person with his shoes off .A boy , an adult and two dogs running through the snow
NEWLINE

NEWLINE
A man stands by a building while others are nearby .
NEWLINE
A woman on a trike and a team of dogs .

NEWLINE
The baby is smiling whilst sitting in a high chair with a red white and blue toy .

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/neham/anaconda3/envs/seathru/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/neham/anaconda3/envs/seathru/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/neham/anaconda3/envs/seathru/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/neham/Desktop/CS6910_Assignments/Assignment_3/dataset.py", line 66, in __getitem__
    print(tokens.shape)
AttributeError: 'list' object has no attribute 'shape'



NEWLINE
A man snowboards over an upturned garbage can in the woods .
NEWLINE
Three young men wearing colorful swim trunks are diving into the water .
NEWLINEThe brown dog is running through the snow
NEWLINE
A man rides a pinkish motorbike on a hill .A snowboarder in a white coat in the air after a jump .

NEWLINENEWLINE

A red tugboat is about to dock at a port .
NEWLINE
A man jumping on his bmx with another bmxer watching .
NEWLINE
Older man in a suit , tie , and overcoat , walks through the street .
NEWLINE
A girl in a pink outfit looks through luggage in a tent city .
NEWLINE
A woman holding a video camera .
NEWLINE
The people are making hand gestures .
NEWLINE
A boy at bat playing a baseball game falls to the ground while onlookers watch .
NEWLINE
Two black and brown dobermans running in a field playing ball .
NEWLINE
In a crowd , two other people finger the fringe of a knit garment hanging on a woman 's shoulder .
NEWLINE
Person doing a trick wheelie on a motorcycle .
NEWLINE
A l